In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
data = pd.read_csv("./data/Churn_Modelling.csv")
data.head()

In [ ]:
## Preprocessing the data
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data.head()

In [ ]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data["Gender"])

In [ ]:
data

In [ ]:
# One hot encoding geography
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse_output=False)
geo_encoder = one_hot_encoder.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
one_hot_encoder.get_feature_names_out()

In [ ]:
geo_df = pd.DataFrame(geo_encoder,columns = one_hot_encoder.get_feature_names_out())
geo_df

In [ ]:
## combine columns
data = pd.concat([data.drop('Geography', axis=1), geo_df],axis=1)
data

In [ ]:
## splitting the data
data.head()

In [ ]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']


## split data into train test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


## scaling 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [15]:
X_test


array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

ANN Regression model implementation

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [32]:
## Build the model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), #connected with input layer
    Dense(32, activation = 'relu'), # hidden layer 2
    Dense(1) # output layer
])

C:\Users\abhis\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


input layer =  12

hidden layer = 64 neurons

hidden layer = 32

output layer = 1

In [33]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# import tensorflow
# opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
# loss = tensorflow.keras.losses.BinaryCrossentropy()

In [35]:
## compiling model
model.compile(optimizer = 'adam', loss = "mean_absolute_error", metrics = ['mae'])


In [36]:
## setting up tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y %m %d - %H %M %S")
tensorflow_callback = TensorBoard(log_dir, histogram_freq=1)

In [37]:
## setting up early stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 75, restore_best_weights = True)

In [38]:
## Trainning the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 100884.1797 - mae: 100884.1797 - val_loss: 98492.3516 - val_mae: 98492.3516
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99127.6484 - mae: 99127.6484 - val_loss: 96829.9922 - val_mae: 96829.9922
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97907.8516 - mae: 97907.8516 - val_loss: 92544.0859 - val_mae: 92544.0859
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 92602.5547 - mae: 92602.5547 - val_loss: 85449.0469 - val_mae: 85449.0469
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 84221.7188 - mae: 84221.7188 - val_loss: 76444.5859 - val_mae: 76444.5859
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75265.7578 - mae: 75265.7578 - val_loss: 67305.9375 - val_mae: 67305.9375
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 65956.2891 - mae: 65956.2891 - val_loss: 59702.7930 - val_mae: 59702.7930
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [ ]:
# model.save('model.h5')

In [40]:
## loading tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6012 (pid 30444), started 0:14:32 ago. (Use '!kill 30444' to kill it.)

In [42]:
# evaluation
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f" Test mae = {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - loss: 51051.7969 - mae: 51051.7969
 Test mae = 50259.4296875


In [43]:
model.save('regression_model.h5')